In [1]:
import json
import torch
from sentence_transformers import util

In [2]:
big = "memoryalpha"
small = "stexpanded"
embeddings = "dogtag_bgelarge"

In [3]:
embeddings_file_big = "./_input/node_embeddings/" + embeddings + "/" + big + ".json"
embeddings_file_small = "./_input/node_embeddings/" + embeddings + "/" + small + ".json"

top10pairs_file_forward = "./_input/top10pairs/" + embeddings + "/" + small + "-" + big + ".json"
top10pairs_file_backward = "./_input/top10pairs/" + embeddings + "/" + big + "-" + small + ".json"

In [4]:
with open(embeddings_file_big, "r") as f:
    embeddings_big = json.load(f)

with open(embeddings_file_small, "r") as f:
    embeddings_small = json.load(f)

In [5]:
small_torch_embeds = torch.Tensor(list(embeddings_small.values()))
big_torch_embeds = torch.Tensor(list(embeddings_big.values()))

pair_top10 = util.semantic_search(small_torch_embeds, big_torch_embeds, top_k=10)
reverse_pair_top10 = util.semantic_search(big_torch_embeds, small_torch_embeds, top_k=10)

In [6]:
forward_dict = dict()
backward_dict = dict()

big_keys = list(embeddings_big.keys())
small_keys = list(embeddings_small.keys())

for a, b in zip(small_keys, pair_top10):
    row_info = list()
    for element in b:
        row_info.append([big_keys[element["corpus_id"]], element["score"]])
    forward_dict[str(a)] = row_info

for a, b in zip(big_keys, reverse_pair_top10):
    row_info = list()
    for element in b:
        row_info.append([small_keys[element["corpus_id"]], element["score"]])
    backward_dict[str(a)] = row_info

with open(top10pairs_file_forward, "w") as f:
    json.dump(forward_dict, f)

with open(top10pairs_file_backward, "w") as f:
    json.dump(backward_dict, f)